# Tutorial 03: weak imposition of Dirichlet BCs by a Lagrange multiplier (interface problem)

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D, using a domain decomposition approach for $\Omega = \Omega_1 \cup \Omega_2$, and introducing a lagrange multiplier to handle the continuity of the solution across
the interface $\Gamma$ between $\Omega_1$ and $\Omega_2$.

The resulting weak formulation is:
$$
\text{find }u_1 \in V(\Omega_1), u_2 \in V(\Omega_2), \eta \in E(\Gamma)
$$
s.t.
$$
\int_{\Omega_1} \nabla u_1 \cdot \nabla v_1 dx +
\int_{\Omega_2} \nabla u_2 \cdot \nabla v_2 dx +
\int_{\Gamma} \lambda (v_1 - v_2) ds = 0,
\qquad \forall v_1 \in V(\Omega_1), v_2 \in V(\Omega_2)
$$
and
$$
\int_{\Gamma} \eta  (u_1 - u_2) ds = 0,
\qquad \forall \eta \in E(\Gamma)
$$
where boundary conditions on $\partial\Omega$ are embedded in $V(\Omega_i) \subset H^1(\Omega_i)$, $i = 1, 2$, and $E(\Gamma) \subset L^2(\Gamma)$.

This example is a prototypical case of problems containing interface restricted variables (the Lagrange multiplier, in this case).

In [ ]:
import numpy as np
from petsc4py import PETSc
from ufl import grad, inner, Measure, TestFunction, TrialFunction
from dolfinx import DirichletBC, Function, FunctionSpace, MPI, solve
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block, BlockVecSubVectorWrapper,
                         create_vector_block, DofMapRestriction, locate_dofs_topological)
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Mesh

In [ ]:
if MPI.size(MPI.comm_world) > 1:
    mesh_ghost_mode = GhostMode.shared_facet  # shared_facet ghost mode is required by dS
else:
    mesh_ghost_mode = GhostMode.none
with XDMFFile(MPI.comm_world, "data/circle.xdmf") as infile:
    mesh = infile.read_mesh(mesh_ghost_mode)
with XDMFFile(MPI.comm_world, "data/circle_subdomains.xdmf") as infile:
    subdomains = infile.read_mf_int(mesh)
with XDMFFile(MPI.comm_world, "data/circle_boundaries.xdmf") as infile:
    boundaries = infile.read_mf_int(mesh)
cells_Omega1 = np.where(subdomains.values == 1)[0]
cells_Omega2 = np.where(subdomains.values == 2)[0]
facets_partial_Omega = np.where(boundaries.values == 1)[0]
facets_Gamma = np.where(boundaries.values == 2)[0]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)
dS = Measure("dS")(subdomain_data=boundaries)
dS = dS(2)  # restrict to the interface, which has facet ID equal to 2

### With domain decomposition

In [ ]:
# Define function spaces
V = FunctionSpace(mesh, ("Lagrange", 2))
V1 = V.clone()
V2 = V.clone()
M = V.clone()

In [ ]:
# Define restrictions
dofs_V1_Omega1 = locate_dofs_topological(V1, subdomains.dim, cells_Omega1)
dofs_V2_Omega2 = locate_dofs_topological(V2, subdomains.dim, cells_Omega2)
dofs_M_Gamma = locate_dofs_topological(M, boundaries.dim, facets_Gamma)
restriction_V1_Omega1 = DofMapRestriction(V1.dofmap, dofs_V1_Omega1)
restriction_V2_Omega2 = DofMapRestriction(V2.dofmap, dofs_V2_Omega2)
restriction_M_Gamma = DofMapRestriction(M.dofmap, dofs_M_Gamma)
restriction = [restriction_V1_Omega1, restriction_V2_Omega2, restriction_M_Gamma]

In [ ]:
# Define trial and test functions
(u1, u2, l) = (TrialFunction(V1), TrialFunction(V2), TrialFunction(M))
(v1, v2, m) = (TestFunction(V1), TestFunction(V2), TestFunction(M))

In [ ]:
# Define problem block forms
zero = Function(V)
a = [[inner(grad(u1), grad(v1)) * dx(1), None, l("-") * v1("-") * dS],
     [None, inner(grad(u2), grad(v2)) * dx(2), - l("+") * v2("+") * dS],
     [m("-") * u1("-") * dS, - m("+") * u2("+") * dS, None]]
f = [v1 * dx(1), v2 * dx(2), zero * m("-") * dS]

In [ ]:
# Define boundary conditions
dofs_V1_partial_Omega = locate_dofs_topological((V1, V), boundaries.dim, facets_partial_Omega)
dofs_V2_partial_Omega = locate_dofs_topological((V2, V), boundaries.dim, facets_partial_Omega)
bc1 = DirichletBC(zero, dofs_V1_partial_Omega, V1)
bc2 = DirichletBC(zero, dofs_V2_partial_Omega, V2)
bcs = [bc1, bc2]

In [ ]:
# Assemble the block linear system
A = assemble_matrix_block(a, bcs=bcs, restriction=(restriction, restriction))
A.assemble()
F = assemble_vector_block(f, a, bcs=bcs, restriction=restriction)

In [ ]:
# Solve
u1u2l = create_vector_block(f, restriction=restriction)
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, u1u2l)
u1u2l.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
(u1, u2, l) = (Function(V1), Function(V2), Function(M))
with BlockVecSubVectorWrapper(u1u2l, [V1.dofmap, V2.dofmap, M.dofmap], restriction) as u1u2l_wrapper:
    for u1u2l_wrapper_local, component in zip(u1u2l_wrapper, (u1, u2, l)):
        with component.vector.localForm() as component_local:
            component_local[:] = u1u2l_wrapper_local

In [ ]:
plot(u1)

In [ ]:
plot(u2)

In [ ]:
plot(l)

### Without domain decomposition

In [ ]:
# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)

In [ ]:
# Define problem forms
a_ex = inner(grad(u), grad(v)) * dx
f_ex = v * dx

In [ ]:
# Define Dirichlet BC object on Gamma
dofs_V_partial_Omega = locate_dofs_topological(V, boundaries.dim, facets_partial_Omega)
bc_ex = DirichletBC(zero, dofs_V_partial_Omega)

In [ ]:
# Solve
u_ex = Function(V)
solve(a_ex == f_ex, u_ex, bc_ex,
      petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
plot(u_ex)

### Comparison and error compuation

In [ ]:
u_ex1_norm = np.sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(u_ex, u_ex) * dx(1))))
u_ex2_norm = np.sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(u_ex, u_ex) * dx(2))))
err1_norm = np.sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(u_ex - u1, u_ex - u1) * dx(1))))
err2_norm = np.sqrt(MPI.sum(mesh.mpi_comm(), assemble_scalar(inner(u_ex - u2, u_ex - u2) * dx(2))))
print("Relative error on subdomain 1", err1_norm / u_ex1_norm)
print("Relative error on subdomain 2", err2_norm / u_ex2_norm)
assert np.isclose(err1_norm / u_ex1_norm, 0., atol=1.e-10)
assert np.isclose(err2_norm / u_ex2_norm, 0., atol=1.e-10)